In [1]:
import pandas as pd
import plotly.express as px
from itertools import product
import numpy as np

In [2]:
df = pd.read_csv(r"data/train.csv")

In [3]:
df.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [4]:
for column in list(df.columns):
    print(f"{column}:\n\t Dtype {df[column].dtype}: null freq {sum(df[column].isnull())/len(df[column])}: unique vals {df[column].nunique()/len(df[column])}".replace("object", "string"))

PassengerId:
	 Dtype string: null freq 0.0: unique vals 1.0
HomePlanet:
	 Dtype string: null freq 0.023122052225928908: unique vals 0.00034510525710341653
CryoSleep:
	 Dtype string: null freq 0.02496261359714713: unique vals 0.0002300701714022777
Cabin:
	 Dtype string: null freq 0.02289198205452663: unique vals 0.7546301621994709
Destination:
	 Dtype string: null freq 0.02093638559760727: unique vals 0.00034510525710341653
Age:
	 Dtype float64: null freq 0.020591280340503854: unique vals 0.009202806856091109
VIP:
	 Dtype string: null freq 0.023352122397331185: unique vals 0.0002300701714022777
RoomService:
	 Dtype float64: null freq 0.02082135051190613: unique vals 0.14643966409754974
FoodCourt:
	 Dtype float64: null freq 0.021051420683308408: unique vals 0.17335787415161624
ShoppingMall:
	 Dtype float64: null freq 0.02392729782583688: unique vals 0.1282641205567698
Spa:
	 Dtype float64: null freq 0.021051420683308408: unique vals 0.15265155872541125
VRDeck:
	 Dtype float64: null freq 

In [5]:
df["CabinClean"] = df.Cabin.str.split("/")
df["CabinCleanNew"] = df["CabinClean"].str[0] + df["CabinClean"].str[2]
df.drop(columns="CabinClean", inplace=True)

In [6]:
lNums = set(df.select_dtypes(include=np.number).columns)
lNums
lStrings = set(df.columns).difference(lNums)
print(lNums)
print(lStrings)

{'ShoppingMall', 'Age', 'RoomService', 'Spa', 'VRDeck', 'FoodCourt'}
{'PassengerId', 'CryoSleep', 'Name', 'Destination', 'Transported', 'HomePlanet', 'CabinCleanNew', 'Cabin', 'VIP'}


In [9]:
df_train_start = df[
    ['Transported', 'VIP', 'HomePlanet', 'Destination','CabinCleanNew',
    'CryoSleep','VRDeck', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Age', 'Spa']
    ].copy()

In [17]:
x_columns = list(df_train_start.columns)
x_columns.remove('Transported')
X = df_train_start[x_columns].copy()
Y = df_train_start["Transported"]
X

,VIP,HomePlanet,Destination,CabinCleanNew,CryoSleep,VRDeck,RoomService,FoodCourt,ShoppingMall,Age,Spa
0,False,Europa,TRAPPIST-1e,BP,False,0.0,0.0,0.0,0.0,39.0,0.0
1,False,Earth,TRAPPIST-1e,FS,False,44.0,109.0,9.0,25.0,24.0,549.0
2,True,Europa,TRAPPIST-1e,AS,False,49.0,43.0,3576.0,0.0,58.0,6715.0
3,False,Europa,TRAPPIST-1e,AS,False,193.0,0.0,1283.0,371.0,33.0,3329.0
4,False,Earth,TRAPPIST-1e,FS,False,2.0,303.0,70.0,151.0,16.0,565.0
...,...,...,...,...,...,...,...,...,...,...,...
8688,True,Europa,55 Cancri e,AP,False,74.0,0.0,6819.0,0.0,41.0,1643.0
8689,False,Earth,PSO J318.5-22,GS,True,0.0,0.0,0.0,0.0,18.0,0.0
8690,False,Earth,TRAPPIST-1e,GS,False,0.0,0.0,0.0,1872.0,26.0,1.0
8691,False,Europa,55 Cancri e,ES,False,3235.0,0.0,1049.0,0.0,32.0,353.0


In [18]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=10,
                                 n_jobs=-1, random_state=42)
rnd_clf.fit(X, Y)

ValueError: could not convert string to float: 'Europa'